Pipeline qui vise a essayer la récupération de triplets a l'aide de rebel et mrebel. Ce fichier a été utilisé pour comparer les inférences des deux modèles et adapter la pipeline ansi que différents tests.

In [ ]:
%%capture
!pip install transformers pyvis pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.9 MB/s eta 0:00:00


In [ ]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import time

# show HTML in notebook
import IPython

# graph visualization
from pyvis.network import Network

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU")


Using Tesla T4


In [ ]:
import requests

# Replace 'your_pdf_url' with the actual URL of your PDF file
pdf_url = 'https://josephbeasse.fr/wp_202320_.pdf'

# Download the PDF
response = requests.get(pdf_url)

# Check if the download was successful
if response.status_code == 200:
    # Save the PDF to a local file in Colab
    with open('wp_202320_.pdf', 'wb') as file:
        file.write(response.content)
        print("PDF Downloaded.")

    # You can now work with the 'downloaded_pdf.pdf' file in your Colab notebook
else:
    print("Failed to download the PDF file.")

PDF Downloaded.


In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 50.9 MB/s eta 0:00:00


In [ ]:
import fitz
def extract_text_from_pdf(pdf_file_path, start_page=0, end_pages_to_skip=0):
    try:
        doc = fitz.open(pdf_file_path)
        text = ""
        total_pages = doc.page_count
        for page_num in range(start_page, total_pages - end_pages_to_skip):
            if page_num < start_page:
                continue
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        return str(e)

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", tgt_lang="tp_XX")  # you can add src_lang= "en_XX" or "fr_XX to add an input language. Makes the model more precise !
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large").to(device)

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

In [ ]:
def extract_relations_from_model_output(text):
    triplets = []
    relation = ''
    text = text.strip()
    current = 'x'
    subject, relation, object_, object_type, subject_type = '','','','',''

    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").split():
        if token == "<triplet>" or token == "<relation>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
    return triplets



In [ ]:
# knowledge base class for small text extraction ONLY
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [ ]:
# build a knowledge base from text
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
    "forced_bos_token_id": None,
    }
    print("Timer started")
    start = time.time()

    generated_tokens = model.generate(
      model_inputs["input_ids"].to(model.device),
      attention_mask=model_inputs["attention_mask"].to(model.device),
      decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
      **gen_kwargs,
    )

    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    print(f"Timer ended: {time.time()-start}")

    for idx, sentence in enumerate(decoded_preds):
      relations = extract_relations_from_model_output(sentence)
      for r in relations:
            kb.add_relation(r)
    return kb

In [ ]:
#@title [RUN THIS CELL] Text is defined here

text = ("Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), "
        "and later known by his regnal name Napoleon I, was a French military and political leader "
        "who rose to prominence during the French Revolution and led several successful campaigns "
        "during the Revolutionary Wars. He became the de facto leader of the French Republic, "
        "serving as First Consul from 1799 to 1804. Assuming the title of Emperor of the French in 1804, "
        "he held this position until 1814 and briefly returned to power in 1815. Napoleon's political "
        "and cultural legacy has endured, marking him as one of the most celebrated and controversial "
        "leaders in world history.\n\n"

        "Napoleon's journey began with his remarkable achievements in the Revolutionary Wars, where his "
        "strategic brilliance and charismatic leadership set him apart. Ascending to the role of First Consul "
        "in 1799, he played a crucial role in navigating France through a period of significant transformation. "
        "His coronation as Emperor in 1804 marked a new chapter, during which he implemented enduring reforms, "
        "including the influential Napoleonic Code.\n\n"

        "The French Empire under Napoleon's rule became a dominant force in Europe, yet his expansionist ambitions "
        "and conflicts with other powers led to his eventual downfall. Following his abdication in 1814 and a brief "
        "exile to Elba, he returned in 1815 for the Hundred Days, only to face ultimate defeat at the Battle of "
        "Waterloo. Subsequently, he spent the remaining years of his life in exile on the remote island of Saint Helena, "
        "where he passed away on 5 May 1821.\n\n"

        "Napoleon's legacy is a subject of ongoing fascination and debate. His impact on military strategy, governance, "
        "and legal systems has left an enduring mark on history, ensuring his place as a complex and influential figure.")

text += ("\n\nOne of Napoleon's pivotal achievements was the establishment of the Napoleonic Code, "
         "a comprehensive legal system that profoundly influenced modern legal principles. Instituted in 1804, "
         "this legal code represented a departure from the complex and disparate laws that existed in France before his reign. "
         "The Napoleonic Code prioritized clarity, consistency, and equality before the law, serving as a foundational document "
         "for legal reform not only in France but also in numerous countries around the world.\n\n"

         "Furthermore, Napoleon's military campaigns redefined the art of warfare. His innovative use of artillery, combined with "
         "strategic brilliance, allowed him to achieve remarkable victories on the battlefield. The Battle of Austerlitz in 1805, "
         "often regarded as one of his masterpieces, showcased his ability to outmaneuver and decisively defeat numerically superior "
         "opponents. This battle earned him the title of the 'Master of the Battlefield' and solidified his reputation as a military genius.\n\n"

         "Napoleon's administrative reforms also played a crucial role in shaping the institutions of the French state. He established "
         "the Banque de France in 1800, centralizing financial control and stabilizing the country's economic system. Additionally, he "
         "undertook extensive infrastructure projects, such as the construction of roads, bridges, and canals, contributing to economic "
         "development and national cohesion.\n\n"

         "In the realm of education, Napoleon implemented the University of France system in 1808, aiming to standardize and elevate "
         "educational standards across the nation. This initiative laid the groundwork for the modern education system in France and emphasized "
         "the importance of knowledge and learning in the pursuit of national progress.\n\n"

         "However, Napoleon's ambitious territorial expansions, while showcasing his military brilliance, also led to prolonged warfare and "
         "strained diplomatic relations with other European powers. The Napoleonic Wars, a series of conflicts that spanned from 1803 to 1815, "
         "ultimately resulted in his downfall and exile.\n\n"

         "Despite the controversies surrounding his reign, Napoleon's impact on the course of history cannot be denied. His achievements, whether "
         "in the fields of law, military strategy, or governance, continue to shape the world we live in today.")

text += ("\n\nNapoleon's strategic vision extended beyond the borders of France, as he sought to create a "
         "continental system to economically weaken his adversaries. The imposition of the Continental Blockade, "
         "aimed at isolating Britain economically, showcased his ambition to exert influence on a global scale. "
         "However, this economic warfare strategy faced challenges and contributed to further conflicts.\n\n"

         "His innovative approach to governance included the establishment of the Legion of Honour in 1802, "
         "a merit-based order that recognized individuals for exceptional achievements. This not only fostered "
         "loyalty among his supporters but also highlighted his desire to reward talent and dedication.\n\n"

         "Napoleon's grand vision extended to the realm of architecture as well. The Arc de Triomphe, a monumental "
         "archway in Paris commissioned in 1806, stands as a symbol of his military victories and the glory of the French "
         "army. It remains an iconic structure, commemorating the triumphs of the French forces under his command.\n\n"

         "Despite his military prowess and numerous achievements, Napoleon faced challenges and setbacks. The ill-fated "
         "Russian Campaign of 1812, marked by the harsh Russian winter and guerrilla warfare, proved to be a turning point. "
         "It weakened the Grande Armée significantly and set the stage for subsequent defeats.\n\n"

         "The Treaty of Fontainebleau in 1814 compelled Napoleon to abdicate and led to his exile to the island of Elba. "
         "Yet, his escape from Elba in 1815, known as the Hundred Days, demonstrated his ability to captivate the public imagination "
         "and rally supporters for one final attempt at power, culminating in the Battle of Waterloo.\n\n"

         "Following his second abdication, Napoleon was exiled to the remote island of Saint Helena, where he spent the remainder "
         "of his life reflecting on his legacy and dictating his memoirs. His death on 5 May 1821 marked the end of an era, but his "
         "impact reverberated through history, leaving an enduring legacy that continues to be studied, debated, and remembered.")

text += ("\n\nNapoleon's complex legacy extends to his impact on art and culture. The Napoleonic era witnessed "
         "the flourishing of the Romantic movement, with artists inspired by the grandeur of his reign. Paintings, literature, "
         "and music from this period often reflect the dramatic events and ideals associated with Napoleon, contributing to the "
         "cultural tapestry of the early 19th century.\n\n"

         "The Napoleonic Code, while revolutionary in its emphasis on equality and legal clarity, also faced criticism for certain "
         "restrictions, particularly regarding women's rights. Despite this, its influence endured, shaping legal systems worldwide and "
         "serving as a precursor to modern legal thought.\n\n"

         "Napoleon's military strategies and tactics continue to be studied in military academies worldwide. His emphasis on mobility, "
         "combined arms tactics, and the use of terrain set a standard for military leaders, leaving a lasting impact on the theory and "
         "practice of warfare.\n\n"

         "In the years following Napoleon's death, his remains were eventually repatriated to France in 1840. The construction of Les Invalides "
         "in Paris, where his tomb now rests, serves as a testament to the enduring fascination and recognition of his historical significance.\n\n"

         "While opinions on Napoleon remain divided, with debates ranging from his authoritarian rule to his contributions to modernization, "
         "there is no denying his profound and lasting impact on the course of history. His legacy stands as a complex tapestry woven with "
         "military triumphs, legal innovations, and cultural influence, making Napoleon Bonaparte a figure whose imprint on the world endures.")

In [ ]:
# SHORT TEXT VERSION
import numpy as np

start_time = time.time()
kb = from_small_text_to_kb(text, verbose=True)
print("Elapsed time for short text retrieval: ", np.round(time.time()-start_time,2)," seconds")
kb.print()

Num tokens: 256
Timer started
Timer ended: 3.590034246444702
Elapsed time for short text retrieval:  3.61  seconds
Relations:
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'date of birth', 'tail': '15 August 1769', 'tail_type': 'date'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'date of death', 'tail': '5 May 1821', 'tail_type': 'date'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'conflict', 'tail': 'French Revolution', 'tail_type': 'eve'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'conflict', 'tail': 'Revolutionary Wars', 'tail_type': 'eve'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'country of citizenship', 'tail': 'French Republic', 'tail_type': 'loc'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'position held', 'tail': 'First Consul', 'tail_type': 'concept'}
  {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'position held', 'tail': 'Emperor of the French', 'tail_type':

In [ ]:
# knowledge base class for meta data collection
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [ ]:
# extract relations for each span and put them together in a knowledge base
def from_text_to_kb(text, span_length=128, verbose=False, kb=KB()):
    # tokenize whole text
    inputs = tokenizer([text], max_length=256, padding=True, truncation=True,  return_tensors = 'pt')

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]

    inputs = {
        "input_ids": torch.stack(tensor_ids).to(device),
        "attention_mask": torch.stack(tensor_masks).to(device)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
      "max_length": 256,
      "length_penalty": 0,
      "num_beams": 3,
      "num_return_sequences": num_return_sequences,
      "forced_bos_token_id": None,
    }


    generated_tokens = model.generate(
      inputs["input_ids"].to(model.device),
      attention_mask=inputs["attention_mask"].to(model.device),
      decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
      **gen_kwargs,
    )

    del inputs, tensor_ids, tensor_masks
    torch.cuda.empty_cache()

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    torch.cuda.empty_cache()

    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        print("Sub part ", i)
        for relation in relations:
            print(relation)
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:
# Text to extract triplets from
text = extract_text_from_pdf('wp_202320_.pdf',start_page=2,end_pages_to_skip=15)
start_time = time.time()
kb = from_text_to_kb(text, verbose=True)
rebel_time = np.round(time.time()-start_time,2)
print("Elapsed time for short text retrieval: ",rebel_time ," seconds")
kb.print()

Input has 256 tokens
Input has 2 spans
Span boundaries are [[0, 128], [128, 256]]
Sub part  0
{'head': 'COVID-19 and Mobility', 'head_type': 'media', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  1
{'head': 'COVID-19 and Mobility', 'head_type': 'cel', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  2
{'head': 'COVID-19 and Mobility', 'head_type': 'concept', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  3
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'country', 'tail': 'France', 'tail_type': 'loc'}
Sub part  4
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'country of origin', 'tail': 'France', 'tail_type': 'loc'}
Sub part  5
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'subclass of', 'tail': 'hospitalisations', 'tail_type': 'concept'}
Elapsed time for short text retrieval:  0.52  seconds
Relations:
  {'head': 'COVID-19 and Mobility', 'he

# TO DO LIST
- Enlever le padding/truncation et max_length du modèle
- Créer une fonction qui permet de séparer les tokens en batch de 1024.
- Une fois que les batchs sont fait, utiliser le modèle sur ce batch et vider le cache après chaque itération:

In [ ]:
def process_batches(text, batch_size=1024):
    # Split the text into batches
    text_batches = [text[i:i+batch_size] for i in range(0, len(text), batch_size)]

    # Process each batch
    for batch in text_batches:
        start_time = time.time()
        kb = from_text_to_kb(batch, verbose=True)
        rebel_time = np.round(time.time()-start_time,2)
        print("Elapsed time for short text retrieval: ",rebel_time ," seconds")
        kb.print()

In [ ]:
start_time = time.time()
process_batches(text)
rebel_time = np.round(time.time()-start_time,2)
print("Elapsed time for short text retrieval: ",rebel_time ," seconds")

Input has 254 tokens
Input has 2 spans
Span boundaries are [[0, 128], [126, 254]]
Sub part  0
{'head': 'COVID-19 and Mobility', 'head_type': 'media', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  1
{'head': 'COVID-19 and Mobility', 'head_type': 'cel', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  2
{'head': 'COVID-19 and Mobility', 'head_type': 'concept', 'type': 'main subject', 'tail': 'COVID-19 propagation', 'tail_type': 'concept'}
Sub part  3
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'country', 'tail': 'France', 'tail_type': 'loc'}
Sub part  4
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'country of origin', 'tail': 'France', 'tail_type': 'loc'}
Sub part  5
{'head': 'COVID-19', 'head_type': 'concept', 'type': 'subclass of', 'tail': 'hospitalisations', 'tail_type': 'concept'}
Elapsed time for short text retrieval:  0.72  seconds
Relations:
  {'head': 'COVID-19 and Mobility', 'he

In [ ]:
from pdfminer.high_level import extract_text
import os, json

def get_files(path):
    """
    Get a list of files with the ".pdf" extension in the specified path.

    Args:
        path (str): The path to the directory containing the files.

    Returns:
        list: A list of file names with the ".pdf" extension.
    """
    files = []
    for file in os.listdir(path):
        if file.endswith(".pdf"):
            files.append(file)
    return files


def store_kb(kb):
    """
    Store the knowledge base (KB) as JSON and RDF files.

    Args:
        kb (KnowledgeBase): The knowledge base object containing the entities and relations.

    Returns:
        bool: True if the KB is successfully stored, False otherwise.
    """
    mode = "w"
    fname = "db"

    # Your existing code to create and populate the RDF graph
    relations = []

    # Iterate over each relation and add it to the relations list
    for relation in kb.relations:
        # check if head, relation_type, and tail are URI safe
        head = relation["head"]
        relation_type = relation["type"]
        tail = relation["tail"]

        # Add the triple to the list
        relation_dict = {
            "head": head,
            "type": relation_type,
            "tail": tail
        }
        relations.append(relation_dict)

    # Save the list of triples to a JSON file
    json_filename = "./" + fname + '_r.json'
    with open(json_filename, 'w') as json_file:
        json.dump(relations, json_file)

    return True


def main() :
    """
    Main function for Knowledge Graph Generation.

    This function allows the user to upload a directory containing PDF files,
    extract text from the files, and generate a knowledge graph based on the extracted text.
    The generated graph is then stored and the execution time is displayed.
    """

    files = get_files(path="./db/")
    if files != [] :
        start_time = time.time()
        kb = KB() #! TODO
        for file in files :
            text = extract_text("./db/"+file)
            for i in range(0, len(text), 1000):
                text_part = text[i:i+1000]
                kb = from_text_to_kb(text_part, verbose=False, kb=kb)

            is_stored = store_kb(kb)
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"storing relations from {file} in {execution_time:.4f} seconds.")

main()


Sub part  0
{'head': 'révolte des vignerons du Midi', 'head_type': 'concept', 'type': 'part of', 'tail': 'révolte Des gueux', 'tail_type': 'concept'}
Sub part  1
{'head': 'révolte des vignerons du Midi', 'head_type': 'eve', 'type': 'part of', 'tail': 'révolte Des gueux', 'tail_type': 'eve'}
Sub part  2
{'head': 'révolte des vignerons du Midi', 'head_type': 'concept', 'type': 'participant', 'tail': 'Emmanuel Petit', 'tail_type': 'per'}
{'head': 'révolte des vignerons du Midi', 'head_type': 'concept', 'type': 'participant', 'tail': 'Christophe Levêque', 'tail_type': 'per'}
Sub part  3
{'head': 'Georges Clemenceau', 'head_type': 'per', 'type': 'position held', 'tail': 'président du Conseil et ministre de l’Intérieur', 'tail_type': 'per'}
Sub part  4
{'head': 'Georges Clemenceau', 'head_type': 'per', 'type': 'position held', 'tail': 'président du Conseil', 'tail_type': 'per'}
Sub part  5
{'head': 'Georges Clemenceau', 'head_type': 'per', 'type': 'position held', 'tail': 'président du Conse